In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Port Alfred,ZA,2022-03-28 23:28:38,-33.5906,26.8910,69.19,78,86,1.99
1,1,Ponca City,US,2022-03-28 23:28:38,36.7070,-97.0856,79.90,37,0,12.66
2,2,Kloulklubed,PW,2022-03-28 23:24:15,7.0419,134.2556,79.30,76,40,12.88
3,3,Geraldton,AU,2022-03-28 23:27:51,-28.7667,114.6000,73.81,78,40,16.11
4,4,Puerto Carreno,CO,2022-03-28 23:28:39,6.1890,-67.4859,76.17,89,100,9.86


In [4]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [19]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [24]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [29]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]



clouds = city_data_df["Cloudiness"]
weights = clouds

wind = city_data_df["Wind Speed"]
weights=wind

# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [30]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [36]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [37]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Ponca City,US,2022-03-28 23:28:38,36.7070,-97.0856,79.90,37,0,12.66
2,2,Kloulklubed,PW,2022-03-28 23:24:15,7.0419,134.2556,79.30,76,40,12.88
4,4,Puerto Carreno,CO,2022-03-28 23:28:39,6.1890,-67.4859,76.17,89,100,9.86
6,6,Rikitea,PF,2022-03-28 23:28:40,-23.1203,-134.9692,78.46,69,78,3.83
7,7,Kapaa,US,2022-03-28 23:28:40,22.0752,-159.3190,79.20,74,40,5.01
10,10,Mount Isa,AU,2022-03-28 23:28:41,-20.7333,139.5000,80.37,26,3,11.50
12,12,Vaini,TO,2022-03-28 23:28:42,-21.2000,-175.2000,82.56,83,75,13.80
13,13,Avarua,CK,2022-03-28 23:28:43,-21.2078,-159.7750,84.25,74,20,9.22
23,23,Puerto Ayora,EC,2022-03-28 23:28:47,-0.7393,-90.3518,80.42,79,94,10.38
36,36,Cabo San Lucas,MX,2022-03-28 23:25:35,22.8909,-109.9124,83.50,26,0,14.97


In [38]:
preferred_cities_df.count()

City_ID       146
City          146
Country       145
Date          146
Lat           146
Lng           146
Max Temp      146
Humidity      146
Cloudiness    146
Wind Speed    146
dtype: int64

In [48]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Ponca City,US,79.90,36.7070,-97.0856,
2,Kloulklubed,PW,79.30,7.0419,134.2556,
4,Puerto Carreno,CO,76.17,6.1890,-67.4859,
6,Rikitea,PF,78.46,-23.1203,-134.9692,
7,Kapaa,US,79.20,22.0752,-159.3190,
10,Mount Isa,AU,80.37,-20.7333,139.5000,
12,Vaini,TO,82.56,-21.2000,-175.2000,
13,Avarua,CK,84.25,-21.2078,-159.7750,
23,Puerto Ayora,EC,80.42,-0.7393,-90.3518,
36,Cabo San Lucas,MX,83.50,22.8909,-109.9124,


In [49]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [51]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
  try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
  except (IndexError):
    print("Hotel not found... skipping.")

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 15)

In [52]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [53]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [54]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [55]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [56]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [57]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))